# Lecture 12:  Numerical Integration and Monte Carlo

**Overview:**
* Numerical integration.
* Simple Monte Carlo integration.
* Importance sampling.

**Next Lecture:**
* Markov chain Monte Carlo and the Metropolis Algorithm.
---

In [ ]:
"""
MONTE-CARLO (Integration)
"Random Sampling"

Numerical Integration:
    |
f(x)| ~~~~~~~
    |  |  |
    |__|__|________
    Reimann sum.
    F = integral(a -->b): f(x)dx
    F ~ Sum:(x = 0 -> n-1): F(x)delx
    
    next step: use triangles instead of rectangles generated from straight line distance between endpoints.
    
    h= (b-a)/N
    
    Trapezoidal Rule:
    A_i = 1/2h[f(a + (i-1)h) + f(a + ih)] 
    F = Sum:(i = 1 -> N): A_i = h[1/2f(1) + 1/2f(b) + Sum:(i = 1 -> N): f(a + ih)]
    
    Simpson Rule: 
    Approximate functioin with curves instead of straight lines. Quadratic function specified over each 3 points.
    Take 3 points at -h,0,h
    f(-h) = Ah**2 - Bh + C
    f(0) = C
    f(h) = Ah**2 + Bh + C
    A = (1/h**2)[1/2(f-h) + f(0) + 1/2f(h)]
    B = 1/2h[f(h)-f(-h)]
    C = f(0)
    inegral(-h -> h): (Ax**2 + Bx + C)dx = 2/3Ah**3 + 2Ch
    = !!!!! 1/3h[f(-h) + 4f(0) + f(h)] !!!!!
    
    To integrate from [a, b]
    Step width h
    a + (2i-2)h
    a + (2i-1)h
    a + 2ih
    A_i = 1/3h[f(1 + (2i-2)h) + 4f(a + 2i-1)h) + f(a + 2ih)]
    
    F = Sum:(i = 1 -> N/2): A_i = 1/3h[f(a) + f(b) + 4*Sum:(odd -> N-1): f(a+ oh) + 2*Sum:(even -> n-2): f(a + ih)]
    
    Trapezoidal Error:
    E = 1/12(h**2)[f'(a)-f'(b)]
    
    Simpson's Error:
    E = 1/90(h**4)[f''(a)-f''(b)]
    
    in d-dimenions
    E ~ h**(a/d)
    
    
    
    <A> = integral:(dp**n * dx**n * e**-B+1 * A) / [integral:(dp**n * dx**n * e**-B+1)]

    Solve using Monte Carlo method!
    |
f(x)| ~~~~~~~
    |    
    |______________    
    a              b 
    Probability that random points land under the curve or outside it.
    
    p = integral(a --> b): f(x)dx / y*(b-a)
    
    F_n = A*(K/N) K = counting number of points < f(x)
    
    Error: p = F/A below, 1-p above
    
    prob of K points below and N-K above
    
    p**k(1-p)**(n-k)
    
    n choose k
    
    p(k) = (n choose k) p**k(1-p)**(n-k) ----> binomial distribution
    
    Mean <K> = np
    Variance<K**2> - <K>**2 = Np(1-p) = NF/A(1-F/A)
    Std = np.sqrt(Variance)
    Error on integral: std*(A/N) = sqrt(F(A-F)/sqrt(N)) ~ N**(-1/2)
    NO D, dimension does not matter!!!
    
    
    A simpler MC method:
    Mean value method
    <f> = 1/(b-a)*integral(a --> b): f(x)dx = F/(b-a)
    OR
    F = (b-a)<f>
    <f> ~ 1/N*(Sum:(i=1 -> n): f(x_i)), x_i chosen at random from a uniform distribution
    
    F = V/N*(Sum:(i=1 -> N) f(x_i))
    
    
    IMPORTANCE SAMPLING:
    I = integral:(0 -> 1): x**-1/2 / (e**x + 1) dx
    Select points x_i non-uniformly from integration region
    <g>_w = integral(a -> b): w(x)g(x)dx / integral(a ->b): w(x)dx
    
    Set g(x) = f(x)/w(x)
    
    <f(x)/w(x)>_w = integral(a->b): w(x)f(x)/w(X)dx / integral(a->b): w(x)dx
                  = integral(a->b): f(x)dx / integral(a->b): w(x)dx
                  = I / integral(a->b): w(x)dx
                  I = <f(x)/w(x)_w * ingetral(a->b):w(x)dx
                  
    p(x) = w(x)/ integral(a->b): w(x)dx
    Sample N random points (x_i)
    non-uniformly such that probability of choosing a x_i in interval x -> x + dx is p(x)dx, then avg. # of samples in 
    is Np(x)dx
    Sum(i = 1 -> N): g(x_i) ~ integral(a->b): Np(x)g(x)dx
    
    <g>_w = integral(a -> b): w(x)g(x)dx / integral(a -> b): w(x)dx
    = integral(a -> b): p(x)g(x)dx
    ~ 1/N*Sum(i = 1 -> N): g(x)
    
    I ~ 1/N * (Sum(i = 1 -> N): f(x_i) / w(x_i)) * integral(a - > b): w(x)dx
"""

In [1]:
%matplotlib notebook
%matplotlib inline
import numpy as np
import math
from scipy import integrate

## Dart Board Estimate of $\pi$

The code in the cell below generates a set of random coordinates inside our unit square and calculates the magnitude of the vector defined by these coordinates.

* Run the code in the cell and call out the number generated for Prof. Plumb to plot on the board.

In [8]:
x = 2 * (np.random.random([1, 2])) - 1
print(np.sqrt(np.sum(x**2)))

0.306903795704


### Functions to integrate, and some exact results

In [3]:
def gaussian(x):
    return np.exp(-x**2)

# A Function that is not well behaved 
def Fermi(x):
    num = 1/np.sqrt(x)
    den = np.exp(x)+1
    return num/den

# area of a unit circle
def sphere(x): 
    """ 
    return 1 if point is inside radius, zero otherwise
    x is a multidimensional vector, must have dimension greater than 1
    """
    r = np.sum(x**2, axis = 1) 
    a = (r<=1).astype(int)
    return 1.0*a

# volume of a hypersphere in n dimensions
hypersphere = lambda r, n: math.pi**(n / 2)/math.gamma(n / 2 + 1)*r**n

# exact integral of a Gaussian
analyticalIntegral = np.sqrt(np.pi)

In [4]:
# Numerically integrate a Gaussian 

def riemannSum(f):
    width = 10.0
    n = 100
    dx = width/n
    x = np.arange(-0.5*width, 0.5*width, width/n)

    return np.sum(f(x)) * dx

def Simpson(f):
    width = 10.0
    n = 100
    dx = width/n
    x = np.arange(-0.5*width, 0.5*width, width/n)
    
    s = (f(-0.5*width)+f(0.5*width)) 
    return (2*f(x[2:-2:2]).sum() + 4*f(x[1:-2:2]).sum()+s) * dx/3

# using Scipy's built in integration schemes
scipyIntegral = integrate.quad(gaussian, -100.0, 100.0)

In [5]:
#np.random.seed(256)
def naiveMonteCarlo(f, limits = [-10,10],d = 1, n_points = 1000, NSamples =100):
    """
    Implement a mean value Monte-Carlo Integration in d dimensions
    
    f is function to integrate, must take an input vector x of dimension d
    
    limits define the range of integration, this function only works for integration ranges that are the same
    all dimensitons
    
    n_points are number of points to sample in domain
    NSamples number of time to repeat integration, decrease statistical noise
    """
    width = np.abs(limits[1] - limits[0])
    samples = np.zeros(NSamples)
    
    for i in range(NSamples):
        x = width * (np.random.random([n_points, d])) + limits[0]
        samples[i] = width**d * np.sum(f(x))/n_points    
    
    return samples.mean(), samples.std()
    

In [6]:
print("Analytical (exact) integral = ", analyticalIntegral)
print("Riemann Sum = ", riemannSum(gaussian))
print("Simpsons Rule = ", Simpson(gaussian))
print("SciPy Integral = ", scipyIntegral)
print("Naive Mean Value Monte Carlo = ", naiveMonteCarlo(gaussian))

Analytical (exact) integral =  1.77245385091
Riemann Sum =  1.7724538509
Simpsons Rule =  1.77245385089
SciPy Integral =  (1.772453850905516, 1.976815268282025e-10)
Naive Mean Value Monte Carlo =  (1.7668750204273076, 0.15721757038645076)


### Tasks
* Use the mean value method Monte Carlo method to estimate the value of $\pi$, (area of unit circle) to a higher accuracy than what was done in the demonstration.
* Use the  Monte Carlo integrator to n-dimensions to find the volume of a hypersphere in 10 dimensions? Compare this results with Simpsons rule and the exact value.
* Can you confirm the error on the MC integration is independent of the number of dimensions?

## Importance sampling Monte Carlo
* Review the importance sampling method below. Do you understand all of the steps?
* Can you modify the method and integrate a 4 dimensional Gaussian function?

In [ ]:
def p_normal(stdev, x): # normal distribution
    s = 1.0 / stdev
    s2 = s**2
    return np.exp(-s2 * x**2) * s / np.sqrt(np.pi)

def importanceSampledMonteCarlo(f, p, NSamples=10):
    n = 10000
    stdev = 1.0
    samples = np.zeros(NSamples)

    for i in range(NSamples):
        # sample random values from a normal distribution
        x = np.random.normal(loc = 0.0, scale = np.sqrt(0.5) * stdev, size = n)
        samples[i] = (f(x) / p(stdev, x)).mean()       
    
    return samples.mean(), samples.std()


In [ ]:
print("Naive Mean Value Monte Carlo = ", naiveMonteCarlo(gaussian))
print("Importance Sampled Monte Carlo = ", importanceSampledMonteCarlo(gaussian, p_normal))